In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
datapath = 'data/modified/'
csv_file_nonet = datapath + 'C1W1 and C1W2 NonNetwork Data.csv'
csv_file_net = datapath + 'C1W1 C1W2 With Network No NickInit.csv'

df_nonet = pd.read_csv(csv_file_nonet)
df_net = pd.read_csv(csv_file_net)

In [34]:
f1 = datapath + "C1W1_nonnetwork.txt"  # wave 1, non-network
f2 = datapath + "C1W2_nonnetwork.txt"  # wave 2, non-network
f3 = datapath + "C1W1_network.txt"  # wave 1, network

varnames, descrs = [], []
with open(f1, 'r') as file:  # read from C1W1
    while (line := file.readline().rstrip()):  # go over each line
        chars = line.split()
        varnames.append(chars[0])
        descrs.append(' '.join(chars[1:]))

In [36]:
mappings = []  # mappings of non-network variables between C1W1 and C1W2
with open(f2, 'r') as file:  # read from C1W2
    i = 0  # keep track of column index of C1W2 vars i.e., line number in f2
    while (line := file.readline().rstrip()):  # go over each line
        chars = line.split()
        descr = ' '.join(chars[1:])
        if descr in descrs:
            mappings.append((descrs.index(descr), i))
        i += 1

In [37]:
print(len(mappings))
mappings

117


[(2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 23),
 (24, 24),
 (26, 25),
 (28, 26),
 (30, 27),
 (31, 28),
 (34, 30),
 (35, 31),
 (36, 32),
 (37, 33),
 (38, 34),
 (39, 35),
 (40, 36),
 (41, 37),
 (42, 38),
 (43, 39),
 (44, 40),
 (45, 41),
 (46, 42),
 (47, 43),
 (48, 44),
 (49, 45),
 (50, 46),
 (51, 47),
 (52, 48),
 (53, 49),
 (54, 50),
 (55, 51),
 (56, 52),
 (57, 53),
 (58, 54),
 (59, 56),
 (60, 57),
 (61, 58),
 (62, 59),
 (63, 60),
 (64, 61),
 (65, 62),
 (66, 63),
 (67, 64),
 (68, 65),
 (69, 66),
 (70, 67),
 (71, 68),
 (72, 69),
 (73, 70),
 (81, 71),
 (82, 72),
 (83, 73),
 (84, 74),
 (85, 75),
 (86, 76),
 (87, 77),
 (74, 78),
 (75, 79),
 (76, 80),
 (77, 81),
 (78, 82),
 (79, 83),
 (80, 84),
 (108, 128),
 (109, 129),
 (110, 130),
 (111, 131),
 (112, 132),
 (113, 133),
 (114, 134),
 (115, 135),
 (116, 136),
 (117, 137),
 (118

Corner cases:
- Q36 is combination of AL6A (for men) and AL6B (for women)
- Exclusive to wave 2: Q70 (how old regularly used marijuana), Q199 (whether been incarcerated in the past 6 months), Q208_R4 (other movement within the Protestant faith)
- DA5-8 == Q100-104 (patterns of using alcohol and drugs)